In [1]:
from utility_programs.read_routines import SAMI

In [2]:
from scipy.spatial import Delaunay
from scipy.interpolate import LinearNDInterpolator, griddata
import numpy as np


from datetime import datetime
from tqdm.auto import tqdm
import xarray as xr

import matplotlib.pyplot as plt

import cartopy.crs as ccrs

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
import interpolate_outputs

In [4]:
samipath = '/petastore/phil/Prasoon_satellite_project/more_runs/20130317/sami-gitm-coupled/'

In [7]:
import importlib

In [56]:
interpolate_outputs = importlib.reload(interpolate_outputs)

In [57]:
interpolate_outputs.do_interpolations(samipath,
                                    out_path='.',
                                      dtime_sim_start='20130316',
                                    cols='edens',
                                      engine=None,
                                    save_delauney=True,
                                    show_progress=True)

attempting to reuse existing triangulation file


Reading in SAMI data:   0%|          | 0/596 [00:00<?, ?it/s]

  0%|          | 0/597 [00:00<?, ?it/s]